In [26]:
!pip install transformers==2.11

In [1]:
import torch
import transformers

In [2]:
transformers.__version__

'2.11.0'

In [3]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig

In [4]:
import os
import torch
#os.chdir('/content/drive/My Drive/Checkpoints')

In [5]:
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
config = BartConfig.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

# run this part to Increase the position embedding from 1024 to 2048

In [6]:
            max_pos=2048
            tokenizer.model_max_length=max_pos
            tokenizer.init_kwargs['model_max_length'] = 2048
            current_max_pos, embed_size = model.model.encoder.embed_positions.weight.shape
            max_pos += 2  # NOTE: RoBERTa has positions 0,1 reserved, so embedding size is max position + 2
            config.max_position_embeddings = max_pos
            assert max_pos > current_max_pos
            new_pos_embed = model.model.encoder.embed_positions.weight.new_empty(max_pos, embed_size)
            k = 2
            step = current_max_pos - 2
            while k < max_pos - 1:
                new_pos_embed[k:(k + step)] = model.model.encoder.embed_positions.weight[2:]
                k += step
            
            model.model.encoder.embed_positions.weight.data = new_pos_embed
            #self.model.model.decoder.embed_positions.weight.data = new_pos_embed
        
        
            print("embedding position size increased to {}".format(config.max_position_embeddings))

embedding position size increased to 2050


## Code to increase the the number of special tokens

In [7]:
            print(len(tokenizer))
            print(model.state_dict()['final_logits_bias'].shape)
            special_tokens_dict = {'additional_special_tokens': ["<T>","<ABS>"]}
            num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
            print(len(tokenizer))
            
            print('We have added', num_added_toks, 'tokens')
            model.resize_token_embeddings(len(tokenizer))
            print(model.state_dict()['final_logits_bias'].shape)
            #assert(self.model.state_dict()["model.model.encoder.embed_tokens.weight"].shape[0]==len(self.tokenizer))
            print("Embedding size increased to {}".format(len(tokenizer)))

50265
torch.Size([1, 50264])
50267
We have added 2 tokens
torch.Size([1, 50267])
Embedding size increased to 50267


In [8]:
checkpoint = torch.load("/Users/byronwallace/code/RoboSum/weights/pl_title_/pl_title_2048.ckpt",map_location="cpu")

In [9]:
cnew={}
for key, value in checkpoint['state_dict'].items():
  cnew[".".join(key.split('.')[1:])]=value

In [10]:
model.load_state_dict(cnew)
#print("model loaded with custom weights from {}".format(args.path))

<All keys matched successfully>

In [11]:
checkpoint['state_dict']

OrderedDict([('model.final_logits_bias',
              tensor([[0., 0., 0.,  ..., 0., 0., 0.]])),
             ('model.model.shared.weight',
              tensor([[-0.0364,  0.1316,  0.1803,  ...,  0.2057,  0.0588, -0.0711],
                      [ 0.0046, -0.0083, -0.0065,  ..., -0.0024,  0.0051,  0.0099],
                      [-0.0462,  0.4619, -0.0623,  ...,  0.1106,  0.0303,  0.0457],
                      ...,
                      [ 0.0057,  0.0183, -0.0052,  ..., -0.0025,  0.0332,  0.0030],
                      [-0.0074,  0.0243,  0.0368,  ...,  0.0020,  0.0149, -0.0071],
                      [-0.0019,  0.0208, -0.0218,  ...,  0.0193, -0.0443,  0.0102]])),
             ('model.model.encoder.embed_tokens.weight',
              tensor([[-0.0364,  0.1316,  0.1803,  ...,  0.2057,  0.0588, -0.0711],
                      [ 0.0046, -0.0083, -0.0065,  ..., -0.0024,  0.0051,  0.0099],
                      [-0.0462,  0.4619, -0.0623,  ...,  0.1106,  0.0303,  0.0457],
                

In [12]:
example_input = '''In both groups, homogeneous by sex, age, neurological grading distribution and length of follow-up, the following parameters were considered: the incidence of ischaemic recurrences during the follow-up period; the characters of the recurrences with particular reference to the fatal stroke; the rate of ischaemic events per year. <T> Long-term follow-up in 257 ICA occlusion: comparison between EIAB-treated and untreated patients. <ABS> Symptomatic occlusions occurred in 72.6 percent of the patients, the reconstructed group (46 patients) having a greater number of symptomatic vessels than the nonreconstructed group (63 patients) (p less than 0.05). <T> Surgical and nonsurgical treatment of total carotid artery occlusion. <ABS> The patients were divided in two groups, one being medically treated with anticoagulants, the other operated on either a carotid artery thrombosis by thromboendarterectomy or occlusion treated with an extracranial-intracranial arterial bypass. <T> Medical versus surgical treatment of patients with cerebrovascular insufficiency. A retrospective comparative study. <ABS> All patients had proximal occlusion of one internal carotid artery or intracranial occlusive disease of the internal carotid or middle cerebral arteries. <T> Long-term assessment of cerebral perfusion following STA-MCA by-pass in patients. <ABS> The incidence of ipsilateral cerebral ischaemia was significantly low in the surgical group. <T> Comparison of the clinical results of STA-MCA anastomosis and the medical treatment in the cerebral low perfusion patients with viable brain tissue. <ABS> Baseline values of the mean blood flow velocity at rest on the affected side were reduced in both groups compared with the contralateral healthy side (group A, 46.0 +/- <T> Evaluation of vasomotor reactivity by transcranial Doppler and acetazolamide test before and after extracranial-intracranial bypass in patients with internal carotid artery occlusion. <ABS> Subgroup analyses indicated that final outcomes for patients with mild to moderate paresis on admission were significantly better in the surgical group than in the non-surgical group (94% vs. 53%, p < 0.01). <T> Superficial temporal artery--middle cerebral artery anastomosis for acute cerebral ischemia: the effect of small augmentation of blood flow. <ABS> Compared to the non-surgical patients mean rCBF at this time was higher over both hemipheres. <T> Measurements of regional cerebral blood flow in patients following superficial temporal artery-middle cerebral artery anastomosis. <ABS> Based on these results in 44 patients, the probability that successful surgery reduces the occurrence of ipsilateral ischemic stroke 1 year later was calculated. <T> Clinical results of extracranial-intracranial bypass surgery in patients with hemodynamic cerebrovascular disease. <ABS> Cerebral perfusion assessed with SPECT scan improved in 88% of patients. <T> STA-MCA bypass for symptomatic carotid occlusion and haemodynamic impairment. <ABS> The average time span from diagnosis of carotid occlusion until death was 4.75 years in the nonsurgical group and 4.52 years in the surgical group. <T> Treatment of the totally occluded carotid artery. <ABS> By an average of 3 years after treatment began, 30 of 49 (61%) reached the same end points. <T> Overall management of vascular lesions considered treatable with extracranial-intracranial bypass: part 1. Internal carotid occlusion. <ABS> Clinical improvement occurred in all groups. <T> Long-term clinical and neurophysiological effects of reconstructive vascular surgery for cerebral ischemia. <ABS> There was no increase in cerebral blood flow in one case with no operation. <T> Evaluation of extracranial-to-intracranial bypass surgery using iodine 123 iodoamphetamine single-photon emission computed tomography. <ABS> Two important subgroups of patients fared substantially worse in the surgical group: those with severe middle-cerebral-artery stenosis (n = 109, Mantel-Haenszel chi-square = 4.74), and those with persistence of ischemic symptoms after an internal-carotid-artery occlusion had been demonstrated (n = 287, chi-square = 4.04). <T> Failure of extracranial-intracranial arterial bypass to reduce the risk of ischemic stroke. Results of an international randomized trial. The EC/IC Bypass Study Group. <ABS> Long-term follow-up showed that there was no significant difference in the outcomes. <T> STA-MCA bypass surgery for internal carotid artery occlusion--comparative follow-up study. <ABS> Perfusion MRI before and after acetazolamide administration compares favourably with (99m)Tc-ECD SPECT for the detection of impaired CVR. <T> Perfusion MRI before and after acetazolamide administration for assessment of cerebrovascular reserve capacity in patients with symptomatic internal carotid artery (ICA) occlusion: comparison with 99mTc-ECD SPECT.'''

In [15]:
inputs = tokenizer.batch_encode_plus([example_input], max_length=2048, return_tensors='pt')
summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=100, early_stopping=True)


In [16]:
print([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids])

[' There is insufficient evidence to support the routine use of EIAB in patients with symptomatic carotid artery occlusion and haemodynamic impairment due to cerebral ischaemia . There is a need for well-designed randomised controlled trials in this area . The results of these trials should be interpreted with caution due to the high risk of bias in some of the trials .']


In [27]:
import sys
sys.path.append("transformers/examples")

In [30]:
from lightning_base import BaseTransformer, add_generic_args, generic_train, get_linear_schedule_with_warmup

In [37]:
import argparse
args = argparse.Namespace(PreLoad_Model=False, adam_epsilon=1e-08, cache_dir='', 
                          config_name='', 
                          data_dir='/Users/byronwallace/code/RoboSum/PubMed_Summary/Data/pl_title/', 
                          do_eval=True, do_predict=True, do_train=False, 
                          eval_batch_size=4, evaluate_during_training=True, 
                          fp16=False, fp16_opt_level='O1', gpu_n=0, 
                          gradient_accumulation_steps=1, learning_rate=3e-05, 
                          max_grad_norm=1.0, max_source_length=1024, max_target_length=110, 
                          model_name_or_path='bart-large', modify_model=False, modify_tokenizer=True, 
                          n_gpu=0, n_tpu_cores=0, num_train_epochs=3, 
                          output_dir='/Users/byronwallace/code/RoboSum/transformers/examples/summarization/pl_title', 
                          path='', seed=42, tokenizer_name='', 
                          train_batch_size=4, warmup_steps=0, weight_decay=0.0)

In [38]:
trainer = generic_train(model, args)

GPU available: False, used: False
INFO:lightning:GPU available: False, used: False
No environment variable for node rank defined. Set as 0.


In [39]:
trainer.test(model)

MisconfigurationException: No `training_step()` method defined. Lightning `Trainer` expects as minimum a `training_step()`, `train_dataloader()` and `configure_optimizers()` to be defined.